**Lesson 2: Bear classification**

In [30]:
from fastai import *
from fastai.vision import *

In [31]:
folders = ["teddy","grizzly","black"]

**Before running the below cell, upload files: teddy.txt, black.txt and grizzly.txt containing the urls for the images by clicking on the Add data button on the right hand side.**

In [32]:
path = Path('/kaggle/working/bears')
path.mkdir(parents=True, exist_ok=True)
!cp ../input/* {path}/

**Make sure internet radio button is on.**

In [33]:
for folder in folders:
    dest = path/folder
    dest.mkdir(parents=True, exist_ok=True)
    file = folder + ".txt"
    download_images(path/file, dest, max_pics=200)

In [35]:
from IPython.display import display, HTML
for c in folders:
    print(c)
    verify_images(path/c, delete=True, max_size=500)

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2, ds_tfms=get_transforms(),size=64, num_workers=0).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

**Edit the slice params based on the plot above**

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(1e-3,1e-2))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.most_confused()

**Cleaning the dataset**

In [ ]:
from fastai.widgets import *

In [ ]:
db = (ImageList.from_folder(path)
                   .split_none()
                   .label_from_folder()
                   .transform(get_transforms(), size=224)
                   .databunch()
     )

In [ ]:
learn_cln = cnn_learner(db, models.resnet34, metrics=error_rate)

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn_cln)

In [ ]:
ImageCleaner(ds, idxs, path)